In [22]:
import pandas as pd
import os
import ast

import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import numpy as np
import pandas as pd

### Load data

In [3]:
filepath = './Review - 1/tracks.csv'
tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])
tracks.head()

C:\Users\sanja\AppData\Local\Temp\ipykernel_18368\991715158.py:2: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,48,49,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103) have mixed types. Specify dtype option on import or set low_memory=False.
  tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])


track_id    album                                                            \
         comments      date_created     date_released engineer favorites id   
2               0  26-11-2008 01:44  05-01-2009 00:00      NaN         4  1   
3               0  26-11-2008 01:44  05-01-2009 00:00      NaN         4  1   
5               0  26-11-2008 01:44  05-01-2009 00:00      NaN         4  1   
10              0  26-11-2008 01:45  06-02-2008 00:00      NaN         4  6   
20              0  26-11-2008 01:45  06-01-2009 00:00      NaN         2  4   

track_id                                                                   \
                                        information listens producer tags   
2                                           <p></p>    6073      NaN   []   
3                                           <p></p>    6073      NaN   []   
5                                           <p></p>    6073      NaN   []   
10                                              NaN   47632      NaN   []   
20        <p> "spiritual songs" from Nicky Cook</p>    2710      NaN   []   

track_id  ... Unnamed: 94_level_0 Unnamed: 95_level_0 Unnamed: 96_level_0  \
          ... Unnamed: 94_level_1 Unnamed: 95_level_1 Unnamed: 96_level_1   
2         ...                 NaN                 NaN                 NaN   
3         ...                 NaN                 NaN                 NaN   
5         ...                 NaN                 NaN                 NaN   
10        ...                 NaN                 NaN                 NaN   
20        ...                 NaN                 NaN                 NaN   

track_id Unnamed: 97_level_0 Unnamed: 98_level_0 Unnamed: 99_level_0  \
         Unnamed: 97_level_1 Unnamed: 98_level_1 Unnamed: 99_level_1   
2                        NaN                 NaN                 NaN   
3                        NaN                 NaN                 NaN   
5                        NaN                 NaN                 NaN   
10                       NaN                 NaN                 NaN   
20                       NaN                 NaN                 NaN   

track_id Unnamed: 100_level_0 Unnamed: 101_level_0 Unnamed: 102_level_0  \
         Unnamed: 100_level_1 Unnamed: 101_level_1 Unnamed: 102_level_1   
2                         NaN                  NaN                  NaN   
3                         NaN                  NaN                  NaN   
5                         NaN                  NaN                  NaN   
10                        NaN                  NaN                  NaN   
20                        NaN                  NaN                  NaN   

track_id Unnamed: 103_level_0  
         Unnamed: 103_level_1  
2                         NaN  
3                         NaN  
5                         NaN  
10                        NaN  
20                        NaN  

[5 rows x 103 columns]

In [4]:
keep_cols = [('set', 'split'),
('set', 'subset'),('track', 'genre_top'), ('track', 'genres')]

tracks = tracks[keep_cols]
tracks.head()

track_id       set             track           
             split  subset genre_top     genres
2         training   small   Hip-Hop       [21]
3         training  medium   Hip-Hop       [21]
5         training   small   Hip-Hop       [21]
10        training   small       Pop       [10]
20        training   large       NaN  [76, 103]

### Look at the MFCC Features

In [6]:
filepath = './Review - 1/features.csv'
features = pd.read_csv(filepath, index_col=0,header=[0, 1, 2], skip_blank_lines=True )

In [10]:
mfcc = features['mfcc']
mfcc.head()

statistics   kurtosis                                                    \
number             01        02        03        04        05        06   
track_id                                                                  
2            3.856789  1.541901  0.000816  0.330728  0.118731 -0.342687   
3            4.296755  1.399977  0.112535 -0.211170  0.032953 -0.023489   
5            2.624517  2.415293  0.440233 -0.782131 -0.771069 -0.724216   
10           5.076893  1.161854  2.095651  1.372743 -0.203574 -0.345354   
20          11.880132  4.085320  0.002655  1.518562  0.181947  0.339014   

statistics                                          ...        std             \
number            07        08        09        10  ...         11         12   
track_id                                            ...                         
2          -0.259252  0.146735  0.410656 -0.162872  ...  10.059609   8.601942   
3           0.150404  0.046454  0.033484 -0.064596  ...   8.771966  10.026867   
5           0.090260  0.152119  0.261731 -0.608905  ...   8.863638   9.581952   
10         -0.529139  0.561974  0.281350 -0.150672  ...   8.289734   7.985110   
20          0.367611  0.069932 -0.016032  0.026012  ...   7.706648   6.498904   

statistics                                                              \
number            13        14        15        16        17        18   
track_id                                                                 
2           9.284250  9.245516  8.520863  8.560472  7.651871  7.246555   
3           6.978541  7.650417  9.600357  7.222888  8.398293  7.285423   
5           8.895723  8.141456  8.201844  7.780963  7.132692  7.539753   
10          7.075400  6.972649  7.071393  7.270959  7.051070  6.928591   
20          6.845514  6.934421  7.019398  6.983841  6.813648  7.520811   

statistics                      
number            19        20  
track_id                        
2           7.077188  7.391859  
3           7.417791  8.777440  
5           8.452527  7.334442  
10          6.430473  6.186294  
20          7.098001  7.032246  

[5 rows x 140 columns]

### PCA Analysis and Visualizations

#### 2 Genres

In [76]:
small = tracks['set', 'subset'] <= 'small'
genre1 = tracks['track', 'genre_top'] == 'Instrumental'
genre2 = tracks['track', 'genre_top'] == 'Hip-Hop'

print(small.shape, genre1.shape, genre2.shape)

X = features.loc[small & (genre1 | genre2), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[small & (genre1 | genre2), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.scatter(X[:,0], X[:,1], c=y, cmap='RdBu', alpha=0.9)
plt.show()
X.shape, y.shape

#### Multiple Genres

In [77]:
small = tracks['set', 'subset'] <= 'small'
genre1 = tracks['track', 'genre_top'] == 'Instrumental'
genre2 = tracks['track', 'genre_top'] == 'Hip-Hop'
genre3 = tracks['track', 'genre_top'] == 'Folk'
genre4 = tracks['track', 'genre_top'] == 'Electronic'

print(small.shape, genre1.shape, genre2.shape)

X = features.loc[small & (genre1 | genre2| genre3| genre4), 'mfcc']
X = skl.decomposition.PCA(n_components=2).fit_transform(X)

y = tracks.loc[small & (genre1 | genre2| genre3| genre4), ('track', 'genre_top')]
y = skl.preprocessing.LabelEncoder().fit_transform(y)

plt.scatter(X[:,0], X[:,1], c=y, cmap='RdBu', alpha=0.9)
plt.show()
X.shape, y.shape

### Break into train, validation and test

In [37]:
from sklearn.model_selection import train_test_split
dataset = pd.read_csv('./Review - 1/fma_small_metadata.csv')
datay = dataset['track.7']
dataset.drop(['track.7'], axis=1, inplace=True)

In [62]:
datax = dataset[['track_id', 'album', 'album.7','track.5', 'track.16']]

In [63]:
X_train, X_test, y_train, y_test = train_test_split(datax, datay, train_size=0.8, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8)

In [54]:
# small = tracks['set', 'subset'] == 'small'

# train = tracks['set', 'split'] == 'training'
# val = tracks['set', 'split'] == 'validation'
# test = tracks['set', 'split'] == 'test'

# y_train = tracks.loc[small & train, ('track', 'genre_top')]
# y_val = tracks.loc[small & val, ('track', 'genre_top')]
# y_test = tracks.loc[small & test, ('track', 'genre_top')]

# X_train_mfcc = features.loc[small & train, 'mfcc']
# X_val_mfcc = features.loc[small & val, 'mfcc']
# X_test_mfcc = features.loc[small & test, 'mfcc']



# # X_train = pd.concat([X_train_mfcc, X_train_tonnetz], axis=1)
# # X_val = pd.concat([X_val_mfcc, X_val_tonnetz], axis=1)

# X_train = X_train_mfcc
# X_val = X_val_mfcc

# print(X_train_mfcc.shape)

# print('{} training examples, {} testing examples'.format(y_train.size, y_val.size))
# print('{} features, {} classes'.format(X_train.shape[1], np.unique(y_train).size))

### Standardize Features and Encode Labels

In [66]:
#Shuffle training features
X_train, y_train = skl.utils.shuffle(X_train, y_train, random_state=42)


scaler = skl.preprocessing.StandardScaler(copy=False)
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

C:\Users\sanja\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:402: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [67]:
## Label encode y - data
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.fit_transform(y_val)
y_test = le.fit_transform(y_test)
le.classes_

array(['Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental',
       'International', 'Pop', 'Rock'], dtype=object)

In [ ]:
# X_train[:1]

In [68]:
y_train[:5]

array([4, 0, 7, 5, 7])

### Build models

In [69]:
from sklearn.metrics import f1_score
from time import time

In [70]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target, y_pred, average='micro', pos_label = 1)


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print("F1 score for val set: {:.4f}.".format(predict_labels(clf, X_val, y_val)))
    print("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))

In [72]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
#from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# TODO: Initialize the three models
clf_A = DecisionTreeClassifier(random_state=10, max_depth =4)
clf_B = SVC()
clf_C = LogisticRegression()
clf_D = RandomForestClassifier(random_state=10, max_depth=30, n_estimators=300, min_samples_leaf=6, min_impurity_decrease=0.0002,
                     class_weight='balanced')

for clf in [ clf_A,clf_B, clf_C, clf_D]:
    print("\n{}: \n".format(clf.__class__.__name__))
    train_predict(clf, X_train, y_train, X_val, y_val)


DecisionTreeClassifier: 

Training a DecisionTreeClassifier using a training set size of 5112. . .
Trained model in 0.0080 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.3343.
Made predictions in 0.0010 seconds.
F1 score for val set: 0.3174.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.3174.

SVC: 

Training a SVC using a training set size of 5112. . .
Trained model in 0.9710 seconds
Made predictions in 2.1674 seconds.
F1 score for training set: 0.3732.
Made predictions in 0.5370 seconds.
F1 score for val set: 0.3385.
Made predictions in 0.5460 seconds.
F1 score for test set: 0.3385.

LogisticRegression: 

Training a LogisticRegression using a training set size of 5112. . .
Trained model in 0.0440 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.2625.
Made predictions in 0.0000 seconds.
F1 score for val set: 0.2549.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.2549.

RandomForestClassifier: 

Training a 

In [73]:
from sklearn.metrics import classification_report

y_true = y_test
y_pred = clf_B.predict(X_test)
labels = [0,1,2,3,4,5,6,7]
target_names = le.inverse_transform(labels)

print(y_true.shape, y_pred.shape)
print(classification_report(y_true, y_pred, target_names=target_names))

(1598,) (1598,)
               precision    recall  f1-score   support

   Electronic       0.00      0.00      0.00       202
 Experimental       0.12      1.00      0.21       188
         Folk       0.00      0.00      0.00       190
      Hip-Hop       0.00      0.00      0.00       194
 Instrumental       0.00      0.00      0.00       211
International       0.00      0.00      0.00       212
          Pop       0.00      0.00      0.00       205
         Rock       0.00      0.00      0.00       196

     accuracy                           0.12      1598
    macro avg       0.01      0.12      0.03      1598
 weighted avg       0.01      0.12      0.02      1598



C:\Users\sanja\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sanja\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sanja\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [74]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))

0.11764705882352941
